<a href="https://colab.research.google.com/github/Prateek-Poonia/Fine-Tuning-BERT/blob/main/Fine_Tune_BERT_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install or force-reinstall the 'pyarrow' package. This is necessary for handling Arrow data formats,
# which are used in conjunction with the 'datasets' library for efficient data loading and processing.
!pip install --force-reinstall pyarrow

# Upgrade the 'datasets' library to the latest version. The 'datasets' library provides tools for easily
# loading and processing datasets, which is crucial for many machine learning tasks.
!pip install --upgrade datasets

# Import TensorFlow, a popular library for building and training machine learning models.
import tensorflow as tf

# Import TFAutoModel and AutoTokenizer from the transformers library.
# TFAutoModel is a TensorFlow-compatible model from Hugging Face's transformers library,
# and AutoTokenizer is used for tokenizing text data so that it can be fed into models.
from transformers import TFAutoModel, AutoTokenizer

# Load a dataset using the 'datasets' library.
# This library provides an easy way to download and load datasets in various formats.
# The 'load_dataset' function can handle different dataset formats and split data into training, validation, and test sets.
from datasets import load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 728.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.33.0 requires numpy<2.0.0,>=1.17, but you have numpy 2.1.1 which is incompatible.
arviz 0.18.0 requires numpy<2.0,>=1.23.0, but you have numpy 2.1.1 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 2.1.1 which is incompatib

In [3]:
# Load a pre-trained TensorFlow model from the Hugging Face Transformers library.
# Here, 'bert-base-uncased' specifies the model architecture (BERT) and the variant (base with uncased text).
# The 'TFAutoModel.from_pretrained' method downloads the model weights and configuration,
# allowing you to use the model for various NLP tasks without training from scratch.
model = TFAutoModel.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
inputs = tokenizer(['Hello world', 'Hi how are you'], padding=True, truncation=True,
                  return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [9]:
# Pass the input data through the pre-trained BERT model to obtain the model's output.
# 'model(inputs)' applies the BERT model to the given 'inputs', which should be tokenized text data.
# The resulting 'output' typically includes hidden states and possibly other model-specific outputs,
# depending on the model and configuration. The exact content of 'output' can vary based on the model architecture.
output = model(inputs)


In [10]:
# Load the 'SetFit/emotion' dataset using the 'datasets' library.
# This dataset is typically used for emotion classification tasks and contains text data labeled with various emotions.
# The 'load_dataset' function downloads and loads the dataset, which can then be used for training and evaluation.
emotions = load_dataset('SetFit/emotion')


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
# Display the contents and structure of the 'emotions' dataset.
# This will show the dataset's splits (e.g., train, test, validation) and a preview of the data within each split.
emotions


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})

In [12]:
def tokenize(batch):
    """
    Tokenize a batch of text data using the pre-trained tokenizer.

    Args:
    batch (dict): A dictionary containing a list of text entries under the key "text".

    Returns:
    dict: A dictionary with tokenized inputs, including input IDs and attention masks,
          with padding and truncation applied as necessary.
    """
    # Use the tokenizer to process the batch of text data.
    # 'batch["text"]' retrieves the list of text entries from the batch dictionary.
    # 'padding=True' ensures that all sequences in the batch are padded to the same length.
    # 'truncation=True' ensures that sequences longer than the model's maximum length are truncated.
    return tokenizer(batch["text"], padding=True, truncation=True)


In [13]:
# Apply the 'tokenize' function to the entire dataset, processing in batches.
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [15]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

emotions_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [16]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
array([[  101,  1045,  3651, ...,     0,     0,     0],
       [  101,  1045,  2572, ...,     0,     0,     0],
       [  101,  1045,  2323, ...,     0,     0,     0],
       ...,
       [  101, 10047,  3110, ...,     0,     0,     0],
       [  101, 10047,  3110, ...,     0,     0,     0],
       [  101,  1045,  2572, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[2 1 0 5 0 0 1 0 4 1 1 0 0 

In [17]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        """
        Initialize the BERT-based classification model.

        Args:
        bert_model (tf.keras.Model): A pre-trained BERT model compatible with TensorFlow.
        num_classes (int): Number of output classes for classification.
        """
        super().__init__()
        # Store the pre-trained BERT model.
        self.bert = bert_model
        # Define a fully connected layer for classification with 'num_classes' output units and softmax activation.
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        """
        Forward pass through the model.

        Args:
        inputs (tf.Tensor): Input tensor(s) to be processed by the BERT model.

        Returns:
        tf.Tensor: The output logits for classification after applying the dense layer.
        """
        # Pass inputs through the BERT model.
        # The BERT model returns a tuple where the second element is the pooled output (representing the [CLS] token).
        x = self.bert(inputs)[1]
        # Apply the dense layer to get classification logits.
        return self.fc(x)


In [18]:
# Instantiate the BERT-based classification model with a specified number of output classes.
# 'model' is the pre-trained BERT model, and 'num_classes=6' specifies there are 6 classes for classification.
classifier = BERTForClassification(model, num_classes=6)

# Compile the model for training.
# - 'optimizer': Adam optimizer with a learning rate of 1e-5.
# - 'loss': SparseCategoricalCrossentropy, appropriate for classification tasks where labels are integers.
# - 'metrics': List of metrics to evaluate during training and evaluation, here it is accuracy.
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)


In [19]:
# Train the BERT-based classification model using the provided training dataset.
# - 'train_dataset': The dataset used for training, which should be properly formatted and tokenized.
# - 'epochs=3': Number of epochs to train the model. This means the model will iterate over the entire dataset 3 times.

history = classifier.fit(
    train_dataset,   # Training data to fit the model
    epochs=3         # Number of training epochs
)


Epoch 1/3
250/250 [==============================] - 316s 1s/step - loss: 1.2186 - accuracy: 0.5247
Epoch 2/3
250/250 [==============================] - 281s 1s/step - loss: 0.3149 - accuracy: 0.8911
Epoch 3/3
250/250 [==============================] - 281s 1s/step - loss: 0.1575 - accuracy: 0.9360


In [20]:
# Evaluate the trained BERT-based classification model on the test dataset.
# - 'test_dataset': The dataset used for evaluation, containing tokenized text and labels.
# This will compute the loss and metrics (e.g., accuracy) on the test data, providing an indication of the model's performance.

evaluation_results = classifier.evaluate(test_dataset)


32/32 [==============================] - 12s 279ms/step - loss: 0.1807 - accuracy: 0.9255
